### See how a model performs on the congruent and incongruent test samples

In [ ]:
import os, sys
import torch
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

sys.path.insert(0, 'src')
from utils.visualizations import show_image_rows, make_grid, plot
from utils import read_lists

import model.metric as module_metric

In [ ]:
dataset_type = '2_Spurious_MNIST'
data_root_dir = os.path.join('data')
train_path = os.path.join(data_root_dir, dataset_type, 'training.pt')
train_data = torch.load(train_path)

test_path = os.path.join(data_root_dir, dataset_type, 'test.pt')
test_data = torch.load(test_path)

### Load colors and labels for test set

In [ ]:
# train_imgs = train_data['images']
# train_labels = train_data['labels']

# test_imgs = test_data['images']
# test_labels = test_data['labels']

n_show = 20
for idx, data in enumerate([train_data, test_data]):
    imgs = data['images']
    labels = data['labels']
    print(imgs[0].shape)
    print(np.amax(imgs[0]))
    show_imgs = imgs[:n_show]
    show_labels = labels[:n_show]
    show_imgs = make_grid(show_imgs, items_per_row=5)
    show_labels = make_grid(show_labels, items_per_row=5)
    show_image_rows(
        images=show_imgs,
        image_titles=show_labels,
        image_size=(1.5, 1.5),
        figure_title='{} {}'.format(dataset_type, 'Train' if idx == 0 else 'Test'))


In [ ]:
dataset_type = '2_Spurious_MNIST'

data_dir = os.path.join('data', dataset_type)
test_data_path = os.path.join(data_dir, 'test.pt')

test_data = torch.load(test_data_path)
test_labels = np.array(test_data['labels'])
test_colors = np.array(test_data['colors'])

congruent_idxs_path = os.path.join(data_dir, 'test_congruent_idxs.pt')
incongruent_idxs_path = os.path.join(data_dir, 'test_incongruent_idxs.pt')

congruent_idxs = torch.load(congruent_idxs_path)
incongruent_idxs = torch.load(incongruent_idxs_path)

In [ ]:
# trial_timestamp = '0317_154335'
# model_arch = 'VGG_16'
# trial_dir = os.path.join('saved', 'edit_{}'.format(dataset_type), '{}-{}'.format(dataset_type, model_arch), trial_timestamp, 'results', 'edit_idx_6')

trial_paths_path = 'saved/edit_2_Spurious_MNIST/method_eac/VGG_16-layernum/0320_111512/trial_paths.txt'
trial_paths = read_lists(trial_paths_path)
trial_dir = os.path.dirname(trial_paths_path)

# trial_logits_path = os.path.join(trial_dir, 'log', 'logits.pth')



# trial_logits_path = os.path.join(trial_dir, 'models', 'post_edit_logits.pth')

# trial_logits = torch.load(trial_logits_path).cpu().numpy()
# trial_predictions = np.argmax(trial_logits, axis=1)


In [ ]:
### Print test set metrics for overall, congruent, and incongruent test set samples

In [ ]:
def print_and_save_partitioned_results(pre_edit_predictions: np.array,
                                       post_edit_predictions: np.array,
                                       labels: np.array,
                                       row_data: dict,
                                       partition_name: str,
                                       metric_fns: list,
                                       mean_only: bool
                                       ):
    print("Calculating {} test set performance".format(partition_name))
    
    metrics['pre'] = module_metric.compute_metrics(
        metric_fns=metric_fns,
        prediction=pre_edit_predictions,
        target=labels,
        unique_labels=[l for l in range(10)],
        save_mean=True)
    
    metrics['post'] = module_metric.compute_metrics(
        metric_fns=metric_fns,
        prediction=post_edit_predictions,
        target=labels,
        unique_labels=[l for l in range(10)],
        save_mean=True)
    
    
    
    for metric_name in metrics['pre'].keys():
        if mean_only and not isinstance(metrics['pre'][metric_name], np.float64):
            continue
        metric_str = "{}: ".format(metric_name)
        # for i in range(n_trials):
        for status in ['pre', 'post']:
            metric_value = metrics[status][metric_name]
            if np.isscalar(metric_value):
                metric_str +="{:.4f} ".format(metric_value)
                row_data['{} {} {}'.format(partition_name, status, metric_name)] = metric_value
            else:
                metric_str +="{} ".format(metric_value)
                row_data[metric_name] = metric_value
            if status == 'pre':
                metric_str += "-> "
        print(metric_str)
    print("")
    
    return row_data
    
def print_summary(congruent_idxs: np.array,
                  incongruent_idxs: np.array,
                  pre_edit_predictions: np.array,
                  post_edit_predictions: np.array, 
                  test_labels: np.array,
                  mean_only=True):
    n_trials = len(trial_predictions)
    row_data = OrderedDict()
    
    partition_labels = ['congruent', 'incongruent']
    metric_names = [
        "accuracy",
        "per_class_accuracy",
        "precision",
        "recall",
        "f1",
        "predicted_class_distribution"]
    metric_fns = [getattr(module_metric, metric_name) for metric_name in metric_names]

    print("Overall test set performance")
    
    row_data = print_and_save_partitioned_results(
        pre_edit_predictions=pre_edit_predictions,
        post_edit_predictions=post_edit_predictions,
        labels=test_labels,
        row_data=row_data,
        partition_name='overall',
        metric_fns=metric_fns,
        mean_only=mean_only)
    
    # Do the same but for congruent/incongruent subsets
    for label_idx, idxs in enumerate([congruent_idxs, incongruent_idxs]):
        congruency_str = 'congruent' if label_idx == 0 else 'incongruent'
        metrics = []
        
        partitioned_labels = test_labels[idxs]
        
        partitioned_pre_edit_predictions = pre_edit_predictions[idxs]
        partitioned_post_edit_predictions = post_edit_predictions[idxs]
        
        row_data = print_and_save_partitioned_results(
            pre_edit_predictions=partitioned_pre_edit_predictions,
            post_edit_predictions=partitioned_post_edit_predictions,
            labels=partitioned_labels,
            row_data=row_data,
            partition_name=congruency_str,
            metric_fns=metric_fns,
            mean_only=mean_only)
    
    return row_data

### Compare pre vs post edit on each partition

In [ ]:
df = pd.DataFrame()
csv_save_path = os.path.join(trial_dir, 'results.csv')
for trial_path in trial_paths:
    # Load pre edit logits & get predictions
    pre_edit_trial_logits_path = os.path.join(trial_path, 'models', 'pre_edit_logits.pth')
    pre_edit_trial_logits = torch.load(pre_edit_trial_logits_path).cpu().numpy()
    pre_edit_trial_predictions = np.argmax(pre_edit_trial_logits, axis=1)

    # Load post edit logits & get predictions
    post_edit_trial_logits_path = os.path.join(trial_path, 'models', 'post_edit_logits.pth')
    post_edit_trial_logits = torch.load(post_edit_trial_logits_path).cpu().numpy()
    post_edit_trial_predictions = np.argmax(post_edit_trial_logits, axis=1)
    
    row_data = OrderedDict()
    row_data['path'] = trial_path
    row_data.update(print_summary(
        congruent_idxs=congruent_idxs,
        incongruent_idxs=incongruent_idxs,
        pre_edit_predictions = pre_edit_trial_predictions,
        post_edit_predictions = post_edit_trial_predictions,
        test_labels=test_labels))
    
    df = df.append(pd.Series(row_data, name=trial_path))

df.set_index('path')

In [ ]:
df.to_csv(csv_save_path)
print("Saved csv to {}".format(csv_save_path))

In [ ]:
# Print layer number vs congruent post accuracy, incongruent post accuracy, and overall post accuracy
plot_save_path = os.path.join(trial_dir, 'layer_v_accuracy.pdf')
labels = ['congruent post accuracy', 'incongruent post accuracy', 'overall post accuracy']
xs = [[i for i in range(1, 13)] for n in range(3)]
ys = [df[label] for label in labels]

plot(
    xs=xs,
    ys=ys,
    labels=labels,
    title='Editing Layer vs Accuracy',
    xlabel='Layer Edited',
    ylabel='Accuracy on Subset',
    save_path=plot_save_path)
